# import

In [1]:
from dotenv import load_dotenv
import os
load_dotenv()
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
ES_HOST = os.getenv("ES_HOST")
ES_USER_ID = os.getenv("ES_ID")
ES_PASSWD = os.getenv("ES_PWD")
ES_INDEX_NAME = os.getenv("ES_INDEX_NAME")
EMBEDDING_BASE_URL=os.getenv("EMBEDDING_BASE_URL")
EMBEDDING_MODEL=os.getenv("EMBEDDING_MODEL")
HUGGINGFACEHUB_API_TOKEN = os.getenv("HUGGUNGFACE_TOKEN")

## Define tools

In [2]:
# from langchain_community.vectorstores.elasticsearch import ElasticsearchStore
from langchain_elasticsearch import ElasticsearchStore, DenseVectorStrategy, ElasticsearchEmbeddings
from elasticsearch import Elasticsearch
from langchain_community.embeddings import HuggingFaceHubEmbeddings
from langchain_openai import ChatOpenAI
from pydantic.v1 import BaseModel, Field  # <-- Uses v1 namespace
from langchain.tools import StructuredTool

In [3]:

class SearchToolInput(BaseModel):
    """Input for the index show data tool."""

    index_name: str = Field(
        ..., description="The name of the index for which the data is to be retrieved. in this case indexname will be " + ES_INDEX_NAME
    )
    query: str = Field(..., description="The ElasticSearch JSON query used to filter all hits. Should use the _source field if possible to specify required fields.")
    from_: int = Field(
        ..., description="The record index from which the query will start"
    )
    size: int = Field(
        ...,
        description="How many records will be retrieved from the ElasticSearch query",
    )

### Embedding 정의

In [4]:
embeddings = HuggingFaceHubEmbeddings(model=EMBEDDING_BASE_URL, huggingfacehub_api_token=HUGGINGFACEHUB_API_TOKEN)

/Users/nuri/dev/jupyter/.venv/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceHubEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEndpointEmbeddings`.
  warn_deprecated(


### ES retirever 정의

In [5]:
es = Elasticsearch([ES_HOST], basic_auth=(ES_USER_ID, ES_PASSWD), ca_certs="./../certs/ca.crt", verify_certs=False, timeout=120)
vector_store = ElasticsearchStore(
    embedding=embeddings,
    index_name=ES_INDEX_NAME,
    es_connection=es,
    strategy=DenseVectorStrategy(hybrid=True)
)
retriever = vector_store.as_retriever(search_kwargs={"k": 10})
# retriever.as_tool()
search_results = retriever.invoke("애플")
print(search_results[0])
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.

/var/folders/jy/5_rl6575533638lskdr9s6s00000gn/T/ipykernel_72063/1781207566.py:1: DeprecationWarning: The 'timeout' parameter is deprecated in favor of 'request_timeout'
  es = Elasticsearch([ES_HOST], basic_auth=(ES_USER_ID, ES_PASSWD), ca_certs="./../certs/ca.crt", verify_certs=False, timeout=120)
/Users/nuri/dev/jupyter/.venv/lib/python3.9/site-packages/elasticsearch/_sync/client/__init__.py:399: SecurityWarning: Connecting to 'https://192.168.0.10:9200' using TLS with verify_certs=False is insecure
  _transport = transport_class(
/Users/nuri/dev/jupyter/.venv/lib/python3.9/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.0.10'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


page_content='"애플 왜 이러나"…전세계서 앱스토어 수시간 먹통 현상
앱스토어를 비롯한 애플의 온라인 플랫폼 서비스들에서 전세계적으로 접속 오류가 나타났다.
이날 접속 장애를 겪은 것은 앱스토어, 애플 아케이드, 애플 북스, 애플 피트니스+, 애플 뮤직, 애플 스포츠, 애플 TV, 애플 팟캐스트 등 대부분의 주요 서비스다.
전세계적으로 접속 장애 현상이 나타난 만큼 한국에서도 4일 오전 8시께 애플 서비스 접속이 불가했다.
이 글자크기로 변경됩니다.
(예시) 가장 빠른 뉴스가 있고 다양한 정보, 쌍방향 소통이 숨쉬는 다음뉴스를 만나보세요. 다음뉴스는 국내외 주요이슈와 실시간 속보, 문화생활 및 다양한 분야의 뉴스를 입체적으로 전달하고 있습니다.
[서울=뉴시스]윤현성 기자 = 앱스토어를 비롯한 애플의 온라인 플랫폼 서비스들에서 전세계적으로 접속 오류가 나타났다. 현재는 문제가 해소됐으나 아직 정확한 원인 등은 파악되지 않았다.
3일(현지시각) CNBC 등 외신에 따르면 애플의 온라인 서비스들은 미국 동부표준시 기준 오후 6시31분부터 오후 7시35분께까지 접속이 불가했다. 이외 지역에서도 1~2시간 가량의 접속 장애 현상이 이어진 것으로 알려졌다.
이날 접속 장애를 겪은 것은 앱스토어, 애플 아케이드, 애플 북스, 애플 피트니스+, 애플 뮤직, 애플 스포츠, 애플 TV, 애플 팟캐스트 등 대부분의 주요 서비스다.' metadata={'id': 540521, 'hash_key': '2f0388b260d53b2a6ef00d9acf56d372', 'title': '"애플 왜 이러나"…전세계서 앱스토어 수시간 먹통 현상', 'created_date': '2024-04-04T11:36:10', 'portal': 'daum', 'media': '뉴시스', 'url': 'https://v.daum.net/v/20240404113610076', 'image_url': 'https://img1.daumcdn.net/thumb/S1200x630/?fname=https:

In [6]:
def create_es_search_tool():
      return StructuredTool(name="elastic_index_search_tool",
                            func=retriever.get_relevant_documents, 
                            args_schema=SearchToolInput)
tools = [create_es_search_tool()]
# tools = [retriever.as_tool()]

In [7]:
# from langchain.tools.retriever import create_retriever_tool

# retrieve_tool = create_retriever_tool(
#     retriever,
#     name="elastic_index_search_tool",
#     description="return search result from user input query"
# )
# tools = [retrieve_tool]

## define llm

In [8]:
llm = ChatOpenAI(model="gpt-4o", api_key=OPENAI_API_KEY)

In [9]:
from langchain_core.messages import HumanMessage

response = llm.invoke([HumanMessage(content="hi!")])
response.content

'Hello! How can I assist you today?'

### agentic

In [10]:
model_with_tools = llm.bind_tools(tools)

In [11]:
response = model_with_tools.invoke([HumanMessage(content=f"{ES_INDEX_NAME} 색인에서 삼성전자 뉴스를 찾아줘!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'elastic_index_search_tool', 'args': {'index_name': 'news_article_embedding', 'query': {'match': {'title': {'query': '삼성전자'}}}, 'from_': 0, 'size': 5}, 'id': 'call_X0wJHduYQAouu437Al2qyKqB', 'type': 'tool_call'}]


# Create Agent

In [12]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(llm, tools)

In [13]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="""
    1. 2024년 애플에 대한 긍정적인 뉴스를 찾아
    2. 찾은 기사에서 중복된 기사는 제거하고. 
    3. 개별 기사에서 긍정적인 사유를 설명하고.
    4. 개별 모든 뉴스에 대해서 지수를 점수로 매겨
    5. 위 결과물을 리스트로 표시해
    """)]}
)
response["messages"]

/Users/nuri/dev/jupyter/.venv/lib/python3.9/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.0.10'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/nuri/dev/jupyter/.venv/lib/python3.9/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.0.10'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/nuri/dev/jupyter/.venv/lib/python3.9/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.0.10'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


[HumanMessage(content='\n    1. 2024년 애플에 대한 긍정적인 뉴스를 찾아\n    2. 찾은 기사에서 중복된 기사는 제거하고. \n    3. 개별 기사에서 긍정적인 사유를 설명하고.\n    4. 개별 모든 뉴스에 대해서 지수를 점수로 매겨\n    5. 위 결과물을 리스트로 표시해\n    ', id='0a0f930e-f087-4955-bbaa-35d05118b3d1'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_ODZreAvPWPWpDXc0NGZ65BU6', 'function': {'arguments': '{"index_name":"news_article_embedding","query":"{\\"query\\":{\\"bool\\":{\\"must\\":[{\\"match\\":{\\"content\\":\\"Apple\\"}},{\\"match\\":{\\"content\\":\\"2024\\"}},{\\"match\\":{\\"content\\":\\"positive\\"}}]}},\\"_source\\":[\\"title\\",\\"content\\"]}","from_":0,"size":10}', 'name': 'elastic_index_search_tool'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 79, 'prompt_tokens': 219, 'total_tokens': 298}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_400f27fa1f', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-89d01386-ce88-449d-971c-c883bd374b22-0', tool_calls=[{'name': 'elas

In [14]:
response["messages"][-1]

AIMessage(content="### 2024년 애플 관련 긍정적인 뉴스 목록\n중복된 기사를 제거하고 개별 기사에서 긍정적인 사유를 설명하며 지수를 매겨 리스트로 정리했습니다.\n\n1. **기사 제목:** 스몰셀 기술로 5G 스마트폰 먹통 막는다\n   - **내용 요약:** 5G 스마트폰의 먹통 문제를 해결하기 위한 스몰셀 기술이 개발되었습니다. 이는 사용자 경험을 대폭 향상시킬 수 있습니다.\n   - **긍정적인 사유:** 5G 기술의 안정성과 사용자 경험 향상.\n   - **지수:** 8/10\n\n2. **기사 제목:** 타인 세포로 만든 면역세포 치료제, 난치성 자가면역질환 치료 성공\n   - **내용 요약:** 타인 세포로 만든 면역세포 치료제가 난치성 자가면역질환 치료에 성공했습니다.\n   - **긍정적인 사유:** 혁신적인 치료법 개발로 인해 다양한 질병 치료 가능성 증가.\n   - **지수:** 9/10\n\n3. **기사 제목:** Gotta catch it all: 'Pokémon Town 2024 with Lotte' coming to Lotte World Tower\n   - **내용 요약:** 롯데월드 타워에 포켓몬 타운이 들어섭니다. 이는 롯데의 자산 가치를 장기적으로 높일 수 있는 사업 모델입니다.\n   - **긍정적인 사유:** 인기 콘텐츠와의 협업으로 고객 유치 및 참여도 증가.\n   - **지수:** 7/10\n\n4. **기사 제목:** AI 학습에 쓴 기사에 돈 내겠다 오픈AI, 독립언론사와 사용 계약\n   - **내용 요약:** 오픈AI가 독립 언론사와 기사 사용 계약을 맺어 AI 학습에 필요한 데이터를 제공받습니다.\n   - **긍정적인 사유:** AI 기술 발전 및 언론사의 수익 창출 기회.\n   - **지수:** 8/10\n\n5. **기사 제목:** 챗GPT 앞서가자 조급했나 구글 '제미나이' 성능 논란\n   - **내용 요약:** 구글의 AI 모델 '제미나이'가 성능 논란을 일으켰습니다